In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data=datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data=datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader=DataLoader(training_data, batch_size=64)
test_dataloader=DataLoader(test_data,batch_size=64)

In [2]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten=nn.Flatten()
        self.linear_relu_stack=nn.Sequential(
            nn.Linear(28*28,512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10)
        )
    def forward(self,x):
        x=self.flatten(x)
        logits=self.linear_relu_stack(x)
        return logits
    
model=NeuralNetwork()

In [3]:
learning_rate=1e-3
batch_size=64
epochs=5

In [4]:
loss_fn=nn.CrossEntropyLoss()

In [6]:
optimizer=torch.optim.SGD(model.parameters(),lr=learning_rate)

In [7]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size=len(dataloader.dataset)
    model.train()
    for batch, (X,y) in enumerate(dataloader):
        pred=model(X)
        loss=loss_fn(pred,y)
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        if batch%100==0:
            loss, current=loss.item(), batch*batch_size + len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [8]:
def test_loop(dataloader, model, loss_fn):
    model.eval()
    size=len(dataloader.dataset)
    num_batches=len(dataloader)
    test_loss, correct=0, 0
    
    with torch.no_grad():
        for X,y in dataloader:
            pred=model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1)==y).type(torch.float).sum().item()
    
    test_loss /=num_batches
    correct /= size
    
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [9]:
loss_fn=nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(model.parameters(),lr=learning_rate)

epochs=10
for t in range(epochs):
    print(f"Epoch {t+1}\n----------------------------")
    train_loop(train_dataloader,model,loss_fn,optimizer)
    test_loop(test_dataloader,model,loss_fn)
print("finito!")

Epoch 1
----------------------------
loss: 2.316299 [   64/60000]
loss: 2.294062 [ 6464/60000]
loss: 2.275429 [12864/60000]
loss: 2.268158 [19264/60000]
loss: 2.236251 [25664/60000]
loss: 2.215839 [32064/60000]
loss: 2.220113 [38464/60000]
loss: 2.183874 [44864/60000]
loss: 2.193262 [51264/60000]
loss: 2.150185 [57664/60000]
Test Error: 
 Accuracy: 45.6%, Avg loss: 2.142328 

Epoch 2
----------------------------
loss: 2.159580 [   64/60000]
loss: 2.143100 [ 6464/60000]
loss: 2.082489 [12864/60000]
loss: 2.106703 [19264/60000]
loss: 2.033705 [25664/60000]
loss: 1.975536 [32064/60000]
loss: 2.012413 [38464/60000]
loss: 1.920391 [44864/60000]
loss: 1.942975 [51264/60000]
loss: 1.870714 [57664/60000]
Test Error: 
 Accuracy: 47.0%, Avg loss: 1.858386 

Epoch 3
----------------------------
loss: 1.894377 [   64/60000]
loss: 1.860836 [ 6464/60000]
loss: 1.740164 [12864/60000]
loss: 1.800714 [19264/60000]
loss: 1.667889 [25664/60000]
loss: 1.622271 [32064/60000]
loss: 1.661401 [38464/60000]
lo